# A1: Data Pseudonymization and Processing Pipeline

**Purpose:** This notebook implements a reproducible pipeline for preparing multi-year Moodle learning management system (LMS) logs for educational data mining analysis. It merges activity logs with student evaluation data while preserving participant anonymity through cryptographic pseudonymization.

## Pipeline Overview

1. **Data Loading**: Import Moodle activity logs and evaluation spreadsheets
2. **Temporal Filtering**: Optionally restrict logs to a specific academic period
3. **Name Matching**: Link records across datasets using exact and fuzzy matching
4. **Data Merging**: Combine activity and evaluation data at the record level
5. **Pseudonymization**: Replace identifiable information with salted cryptographic hashes
6. **Export**: Generate analysis-ready, privacy-preserving datasets

## Privacy Safeguards

- Student identities are replaced with salted SHA-256 hashes (non-reversible)
- Teachers receive deterministic labels without exposing real names
- Sensitive columns (IP addresses, raw names) are dropped before export
- The salt value is never logged or saved to output files

## 1. Environment Setup

The following cell imports all required libraries:

| Library | Purpose |
|---------|--------|
| `pandas` | Data manipulation and CSV I/O |
| `hashlib` | SHA-256 hashing for pseudonymization |
| `datetime` | Date parsing and temporal filtering |
| `difflib.SequenceMatcher` | String similarity computation |
| `re` | Regular expression-based text cleaning |
| `unicodedata` | Unicode normalization (accent removal) |
| `collections.Counter` | Tracking filtered records |

In [1]:
import hashlib
import re
import unicodedata
from collections import Counter
from datetime import datetime
from difflib import SequenceMatcher
from pathlib import Path

import pandas as pd

## 2. Helper Functions

This section defines reusable utility functions organized into three categories:

### 2.1 Name Normalization
Functions for cleaning and standardizing names to enable robust matching across datasets with inconsistent formatting.

### 2.2 Name Matching
Functions implementing the similarity-based matching algorithm that combines sequence alignment with set-based (Jaccard) comparison.

### 2.3 Data Transformation
Functions for pseudonymization, date parsing, and attendance calculation.

### 2.1 Name Normalization Functions

These functions handle the variability commonly found in name data:
- Diacritical marks (accents) that may be present or absent
- Inconsistent capitalization
- Extra whitespace
- Name particles (e.g., "de", "la") that add noise to matching

In [2]:
def strip_accents(text: str) -> str:
    """
    Remove diacritical marks while preserving base characters.
    
    Uses Unicode NFKD normalization to decompose characters into base
    characters and combining marks, then removes the combining marks.
    
    Parameters
    ----------
    text : str
        Input string potentially containing accented characters.
    
    Returns
    -------
    str
        String with diacritical marks removed (e.g., 'GARCÍA' -> 'GARCIA').
    
    Examples
    --------
    >>> strip_accents('José María')
    'Jose Maria'
    """
    normalized = unicodedata.normalize('NFKD', text)
    return ''.join(char for char in normalized if not unicodedata.combining(char))


def clean_name(name) -> str:
    """
    Normalize a name for comparison: trim, uppercase, collapse spaces, remove accents.
    
    Parameters
    ----------
    name : str or None
        Raw name string from input data.
    
    Returns
    -------
    str
        Cleaned, uppercase name with single spaces and no accents.
        Returns empty string for null/NaN input.
    
    Examples
    --------
    >>> clean_name('  María   García  ')
    'MARIA GARCIA'
    """
    if pd.isna(name):
        return ""
    cleaned = str(name).strip().upper()
    cleaned = strip_accents(cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned


def normalize_name_parts(name) -> set:
    """
    Extract meaningful name tokens as an unordered set.
    
    Removes common Spanish name particles and single-character tokens
    to focus on substantive name components. The set representation
    allows order-independent comparison.
    
    Parameters
    ----------
    name : str or None
        Input name string.
    
    Returns
    -------
    set
        Set of normalized name tokens (uppercase, no accents).
    
    Examples
    --------
    >>> normalize_name_parts('María de la Cruz García')
    {'MARIA', 'CRUZ', 'GARCIA'}
    """
    if pd.isna(name) or str(name).strip() == "":
        return set()
    
    cleaned = clean_name(name)
    # Common Spanish name particles to exclude
    particles = {'DE', 'LA', 'LAS', 'DEL', 'LOS', 'Y'}
    words = [w for w in cleaned.split() if w not in particles and len(w) > 1]
    return set(words)


def is_system_account(name) -> bool:
    """
    Identify non-student accounts (administrators, guests, system users).
    
    These accounts should be excluded from student-focused analyses
    and from the name matching process.
    
    Parameters
    ----------
    name : str or None
        Account name to check.
    
    Returns
    -------
    bool
        True if the name indicates a system/admin account.
    """
    if pd.isna(name):
        return False
    
    name_clean = clean_name(name)
    system_indicators = [
        'ADMINISTRADOR', 'ADMIN', 'SYSTEM', 'STUDIUM',
        'USUARIO', 'USER', 'INVITADO', 'GUEST'
    ]
    
    if name_clean == '-':
        return True
    if name_clean in system_indicators:
        return True
    if any(indicator in name_clean for indicator in system_indicators):
        return True
    
    return False

### 2.2 Name Matching Functions

The matching algorithm uses a hybrid similarity measure that combines:

1. **Sequence Similarity (30% weight)**: Uses `difflib.SequenceMatcher` to measure character-level alignment. Good for catching typos and minor variations.

2. **Jaccard Similarity (70% weight)**: Compares the set of name tokens between two names. This is robust to word order differences (e.g., "García López" vs "López García").

The higher weight on Jaccard similarity reflects the common occurrence of name component reordering in our data sources.

In [3]:
def calculate_name_similarity(name1, name2) -> float:
    """
    Compute hybrid similarity score between two names.
    
    Combines character-level sequence similarity (30%) with token-level
    Jaccard similarity (70%) to handle both typos and word reordering.
    
    Parameters
    ----------
    name1 : str
        First name to compare.
    name2 : str
        Second name to compare.
    
    Returns
    -------
    float
        Similarity score in range [0.0, 1.0].
    
    Examples
    --------
    >>> calculate_name_similarity('Juan García López', 'Juan López García')
    0.85  # High score despite word reordering
    """
    # Character-level sequence similarity
    clean1, clean2 = clean_name(name1), clean_name(name2)
    sequence_sim = SequenceMatcher(None, clean1, clean2).ratio()
    
    # Token-level Jaccard similarity
    tokens1 = normalize_name_parts(name1)
    tokens2 = normalize_name_parts(name2)
    
    if not tokens1 or not tokens2:
        return sequence_sim
    
    intersection = len(tokens1 & tokens2)
    union = len(tokens1 | tokens2)
    jaccard_sim = intersection / union if union > 0 else 0.0
    
    # Weighted combination: favor token matching for robustness to reordering
    return 0.3 * sequence_sim + 0.7 * jaccard_sim


def find_best_match(target_name, candidate_names, threshold=0.6):
    """
    Find the best matching name from a list of candidates.
    
    Parameters
    ----------
    target_name : str
        Name to find a match for.
    candidate_names : list of str
        List of potential matching names.
    threshold : float, default=0.6
        Minimum similarity score required for a valid match.
    
    Returns
    -------
    tuple (str or None, float)
        Best matching name and its similarity score.
        Returns (None, 0.0) if no match exceeds the threshold.
    """
    if is_system_account(target_name):
        return None, 0.0
    
    best_match = None
    best_score = 0.0
    
    for candidate in candidate_names:
        if is_system_account(candidate):
            continue
        
        score = calculate_name_similarity(target_name, candidate)
        if score >= threshold and score > best_score:
            best_match = candidate
            best_score = score
    
    return best_match, best_score

### 2.3 Data Transformation Functions

These functions handle:

- **Pseudonymization**: Generating non-reversible identifiers using salted SHA-256 hashing
- **Date Parsing**: Converting Moodle's date format strings to datetime objects
- **Attendance Calculation**: Parsing various attendance formats into percentages

In [4]:
def create_pseudonym(name, salt=None) -> str:
    """
    Generate a pseudonym using salted SHA-256 hashing.
    
    The salt prevents dictionary-based re-identification attacks.
    The same name with the same salt always produces the same pseudonym,
    enabling consistent identification across datasets.
    
    Parameters
    ----------
    name : str
        Original name to pseudonymize.
    salt : str, optional
        Secret salt value. Strongly recommended for privacy protection.
    
    Returns
    -------
    str
        Pseudonym in format 'STU_XXXXXXXX' (8 hex characters).
    """
    base = clean_name(name)
    key = (salt or "") + "|" + base
    hash_value = hashlib.sha256(key.encode("utf-8")).hexdigest()[:8].upper()
    return f"STU_{hash_value}"


def parse_date(date_str):
    """
    Parse Moodle date format strings to datetime objects.
    
    Supports formats:
    - 'DD/MM/YYYY'
    - 'DD/MM/YYYY HH:MM'
    
    Parameters
    ----------
    date_str : str or None
        Date string from Moodle logs.
    
    Returns
    -------
    datetime or None
        Parsed datetime object, or None if parsing fails.
    """
    if pd.isna(date_str):
        return None
    
    date_string = str(date_str).strip()
    
    # Try date-only format first
    try:
        date_part = date_string.split()[0]
        return datetime.strptime(date_part, "%d/%m/%Y")
    except (ValueError, IndexError):
        pass
    
    # Try full datetime format
    try:
        return datetime.strptime(date_string, "%d/%m/%Y %H:%M")
    except ValueError:
        return None


def parse_attendance(value, total_sessions) -> float:
    """
    Parse attendance data into a percentage.
    
    Handles multiple input formats:
    - Integer count (e.g., 24 sessions attended)
    - Fraction string (e.g., '24 de 30' or '24/30')
    - Pre-computed percentage
    
    Parameters
    ----------
    value : int, float, or str
        Attendance value in any supported format.
    total_sessions : int
        Total number of sessions in the course.
    
    Returns
    -------
    float
        Attendance percentage in range [0, 100], or NaN if not computable.
    """
    if pd.isna(value):
        return float('nan')
    
    # Handle numeric values
    if isinstance(value, (int, float)):
        # If value looks like a count (<= total_sessions), convert to percentage
        if total_sessions and value <= total_sessions:
            return (value / total_sessions) * 100.0
        # Otherwise assume it's already a percentage
        return float(value)
    
    # Handle string values
    text = str(value)
    numbers = [int(x) for x in re.findall(r'\d+', text)]
    
    if len(numbers) == 0:
        return float('nan')
    
    if len(numbers) == 1:
        # Single number: treat as attended count
        attended = numbers[0]
        if total_sessions:
            return (attended / total_sessions) * 100.0
        return float('nan')
    
    # Two numbers: assume format is "attended / total"
    attended, total = numbers[0], numbers[1]
    if total == 0:
        return float('nan')
    return (attended / total) * 100.0

## 3. Main Processing Function

The `merge_moodle_evaluation_data` function implements the complete data processing pipeline. It performs the following steps:

| Step | Description |
|------|-------------|
| 1 | Load Moodle logs and evaluation data from CSV files |
| 2 | Validate required columns exist in both datasets |
| 3 | Apply temporal filter to restrict logs to the academic period |
| 4 | Construct normalized full names for matching |
| 5 | Match names in three phases: exact → fuzzy → manual overrides |
| 6 | Filter out unmatched records (non-students, non-teachers) |
| 7 | Merge evaluation fields onto Moodle log records |
| 8 | Pseudonymize identities (deterministic teacher labels, salted student hashes) |
| 9 | Drop sensitive columns and compute derived fields |

The function returns both the processed DataFrame and the name-to-pseudonym mapping (kept in memory only, never written to disk).

In [5]:
# -------------------------------------------------------------------------
# Helper to mask names in console logs (not used in the actual data)
# -------------------------------------------------------------------------
def make_log_name_masker():
    """
    Returns a function that gives each real name a stable alias
    for console display only. Aliases are not stored or written to disk.
    """
    mapping = {}
    
    def mask(name):
        if name is None:
            return "<NULL>"
        if name not in mapping:
            mapping[name] = f"ALIAS_{len(mapping) + 1}"
        return mapping[name]
    
    return mask
    
def merge_moodle_evaluation_data(
    moodle_file,
    evaluation_file,
    total_sessions=30,
    limit_date=None,
    teachers=None,
    output_file=None,
    manual_matches=None,
    fuzzy_threshold=0.6,
    suggestion_threshold=0.5,
    salt=None
):
    """
    Merge Moodle logs with evaluation data and pseudonymize identities.
    
    This function implements a complete ETL pipeline for preparing
    educational data mining datasets while preserving participant privacy.
    
    Parameters
    ----------
    moodle_file : str
        Path to Moodle activity logs CSV file.
    evaluation_file : str
        Path to student evaluation data CSV file.
    total_sessions : int, default=30
        Total number of class sessions (for attendance percentage calculation).
    limit_date : str, optional
        Cutoff date in 'DD/MM/YYYY' format. Moodle actions after this
        date are excluded from the output.
    teachers : list of str, optional
        List of teacher names for identification. Matching is
        case-insensitive and accent-insensitive.
    output_file : str, optional
        If provided, write the merged CSV to this path.
    manual_matches : dict, optional
        Manual name mapping {moodle_name: eval_name} for edge cases.
        These do not override automatic matches.
    fuzzy_threshold : float, default=0.6
        Minimum similarity score for automatic fuzzy matching.
    suggestion_threshold : float, default=0.5
        Minimum similarity for printing match suggestions to console.
    salt : str, optional
        Secret salt for pseudonym hashing. Strongly recommended.
    
    Returns
    -------
    merged_df : pd.DataFrame
        Merged and pseudonymized dataset.
    pseudonym_mapping : dict
        Mapping from original names to pseudonyms (for verification only).
    
    Notes
    -----
    The pseudonym_mapping should be kept confidential and not saved to disk
    to prevent re-identification of participants.
    """

    log_mask = make_log_name_masker()
    
    # Default teacher list (customize for your institution)
    if teachers is None:
        teachers = [
            "Alicia García Holgado",
            "Andrea Vázquez Ingelmo",
            "Francisco José García Peñalvo"
        ]
    
    # =========================================================================
    # STEP 1: Load datasets
    # =========================================================================
    print("Loading datasets...")
    moodle_df = pd.read_csv(moodle_file)
    eval_df = pd.read_csv(evaluation_file)
    
    # =========================================================================
    # STEP 2: Validate required columns
    # =========================================================================
    required_moodle = {'Nombre completo del usuario', 'Hora'}
    required_eval = {'Nombre', 'Apellidos'}
    
    missing_cols = (
        (required_moodle - set(moodle_df.columns)) |
        (required_eval - set(eval_df.columns))
    )
    if missing_cols:
        raise ValueError(f"Missing required columns: {sorted(missing_cols)}")
    
    # =========================================================================
    # STEP 3: Apply temporal filter
    # =========================================================================
    if limit_date:
        limit_dt = datetime.strptime(limit_date, "%d/%m/%Y")
        print(f"Filtering Moodle actions after {limit_date} (inclusive end).")
        
        moodle_df['__parsed_date'] = moodle_df['Hora'].apply(parse_date)
        rows_before = len(moodle_df)
        
        moodle_df = moodle_df[
            (moodle_df['__parsed_date'].isna()) |
            (moodle_df['__parsed_date'] <= limit_dt)
        ].drop(columns='__parsed_date')
        
        print(f"Moodle rows kept: {len(moodle_df)} / {rows_before}")
    
    # =========================================================================
    # STEP 4: Prepare evaluation data with normalized full names
    # =========================================================================
    eval_columns_expected = [
        'Apellidos', 'Nombre',
        'Hito 1', 'Hito 2', 'Hito 3', 'Trabajo',
        'Coevaluación 1', 'Coevaluación 2', 'Coevaluación 3',
        'Asistencia'
    ]
    present_eval_cols = [c for c in eval_columns_expected if c in eval_df.columns]
    eval_subset = eval_df[present_eval_cols].copy()
    
    # Construct full name: "FirstName LastName" in uppercase
    eval_subset['Nombre_completo'] = (
        eval_df['Nombre'].astype(str).str.strip() + ' ' +
        eval_df['Apellidos'].astype(str).str.strip()
    ).str.upper().str.strip()
    
    # =========================================================================
    # STEP 5: Name matching (three phases)
    # =========================================================================
    moodle_names = [
        n for n in moodle_df['Nombre completo del usuario'].dropna().unique()
        if not is_system_account(n)
    ]
    eval_names = eval_subset['Nombre_completo'].dropna().unique().tolist()
    
    print(f"Unique names in Moodle: {len(moodle_names)}")
    print(f"Unique names in Evaluation: {len(eval_names)}")
    
    name_mapping = {}
    
    # Phase 5a: Exact matches (O(N+M) via dictionary lookup)
    eval_clean_to_orig = {clean_name(en): en for en in eval_names}
    for mname in moodle_names:
        mc = clean_name(mname)
        if mc and mc in eval_clean_to_orig:
            name_mapping[mname] = eval_clean_to_orig[mc]
    
    print(f"Exact matches found: {len(name_mapping)}")
    
    # Phase 5b: Fuzzy matches for remaining names
    unmatched_moodle = [n for n in moodle_names if n not in name_mapping]
    matched_eval = set(name_mapping.values())
    unmatched_eval = [n for n in eval_names if n not in matched_eval]
    
    print("Finding fuzzy matches...")
    for mname in unmatched_moodle:
        best, score = find_best_match(mname, unmatched_eval, threshold=fuzzy_threshold)
        if best:
            name_mapping[mname] = best
            unmatched_eval.remove(best)  # Prevent duplicate matches
            print(
                "Fuzzy match: "
                f"'{log_mask(mname)}' -> '{log_mask(best)}' "
                f"(score: {score:.3f})"
            )
    
    print(f"Total matches found after fuzzy: {len(name_mapping)}")
    
    # Phase 5c: Apply manual matches (do not override automatic matches)
    if manual_matches:
        print("Applying manual matches...")
        for mname, ename in manual_matches.items():
            if mname in moodle_names and ename in eval_names:
                if mname not in name_mapping:
                    name_mapping[mname] = ename
                    print(f"Manual match: '{log_mask(mname)}' -> '{log_mask(ename)}'")
                else:
                    print(f"Manual match skipped (already matched): '{log_mask(mname)}'")
            else:
                print(f"Manual match ignored (name not found): '{log_mask(mname)}' -> '{log_mask(ename)}'")
    
    # Report unmatched names and suggest potential matches
    final_unmatched_moodle = [n for n in moodle_names if n not in name_mapping]
    remaining_unmatched_eval = [n for n in eval_names if n not in name_mapping.values()]
    
    if final_unmatched_moodle:
        masked = [log_mask(n) for n in final_unmatched_moodle]
        print(
            f"\nUnmatched in Moodle logs ({len(final_unmatched_moodle)}): "
            f"{masked}"
        )
    if remaining_unmatched_eval:
        masked = [log_mask(n) for n in remaining_unmatched_eval]
        print(
            f"\nUnmatched in Moodle logs ({len(remaining_unmatched_eval)}): "
            f"{masked}"
        )
    
    # Print suggestions for manual review (not automatically applied)
    if final_unmatched_moodle and remaining_unmatched_eval:
        print(f"\nSuggested matches (>={suggestion_threshold:.0%} similarity):")
        remaining_set = set(remaining_unmatched_eval)
        for mname in final_unmatched_moodle:
            best, best_score = None, 0.0
            for ename in list(remaining_set):
                score = calculate_name_similarity(mname, ename)
                if score >= suggestion_threshold and score > best_score:
                    best, best_score = ename, score
            if best is not None:
                print(
                    f"  '{log_mask(mname)}' <-> '{log_mask(best)}' "
                    f"(score: {best_score:.3f})"
                )
    
    # =========================================================================
    # STEP 6: Filter Moodle rows (keep only teachers and matched students)
    # =========================================================================
    print("\nFiltering unmatched non-teacher records...")
    print(f"Original Moodle records: {len(moodle_df)}")
    
    teachers_clean = {clean_name(t) for t in teachers}
    filtered_counts = Counter()
    
    def keep_record(moodle_name):
        if pd.isna(moodle_name):
            filtered_counts["<NaN>"] += 1
            return False
        if clean_name(moodle_name) in teachers_clean:
            return True
        if moodle_name in name_mapping:
            return True
        filtered_counts[moodle_name] += 1
        return False
    
    moodle_df = moodle_df[
        moodle_df['Nombre completo del usuario'].apply(keep_record)
    ]
    
    # Mask names
    masked_counts = {
        ("<NaN>" if k == "<NaN>" else log_mask(k)): v
        for k, v in filtered_counts.items()
    }
    
    print(f"Records removed per user (masked): {masked_counts}")
    print(f"Filtered Moodle records: {len(moodle_df)}")
    
    # =========================================================================
    # STEP 7: Merge evaluation fields onto Moodle records
    # =========================================================================
    print("\nMerging datasets...")
    merged_rows = []
    
    eval_by_name = (
        eval_subset.set_index('Nombre_completo')
        if 'Nombre_completo' in eval_subset.columns
        else pd.DataFrame()
    )
    
    maybe_eval_cols = [
        c for c in [
            'Hito 1', 'Hito 2', 'Hito 3', 'Trabajo',
            'Coevaluación 1', 'Coevaluación 2', 'Coevaluación 3', 'Asistencia'
        ] if c in eval_subset.columns
    ]
    
    for _, mrow in moodle_df.iterrows():
        full_name = mrow['Nombre completo del usuario']
        is_teacher = clean_name(full_name) in teachers_clean
        
        out = mrow.to_dict()
        out['Es_profesor'] = bool(is_teacher)
        
        # Add evaluation data for matched students
        if not is_teacher and full_name in name_mapping and not eval_by_name.empty:
            eval_name = name_mapping[full_name]
            if eval_name in eval_by_name.index:
                row_eval = eval_by_name.loc[eval_name]
                for c in maybe_eval_cols:
                    out[c] = row_eval[c]
            else:
                for c in maybe_eval_cols:
                    out[c] = pd.NA
        else:
            for c in maybe_eval_cols:
                out[c] = pd.NA
        
        merged_rows.append(out)
    
    merged_df = pd.DataFrame(merged_rows)
    
    # =========================================================================
    # STEP 8: Pseudonymization
    # =========================================================================
    print("Pseudonymizing names...")
    original_names = merged_df['Nombre completo del usuario'].dropna().unique().tolist()
    
    # Assign deterministic teacher labels (sorted for reproducibility)
    teacher_names_present = sorted(
        [n for n in original_names if clean_name(n) in teachers_clean],
        key=lambda x: clean_name(x)
    )
    
    pseudonym_mapping = {}
    for idx, tname in enumerate(teacher_names_present, start=1):
        pseudonym_mapping[tname] = f"TEACHER_{idx}"
    
    # Generate salted hashes for students
    for name in original_names:
        if name not in pseudonym_mapping:
            pseudonym_mapping[name] = create_pseudonym(name, salt=salt)
    
    merged_df['Nombre_pseudonimo'] = merged_df['Nombre completo del usuario'].map(
        pseudonym_mapping
    )
    
    # =========================================================================
    # STEP 9: Compute attendance percentage and finalize
    # =========================================================================
    if 'Asistencia' in merged_df.columns:
        merged_df['% Asistencia'] = merged_df['Asistencia'].apply(
            lambda v: parse_attendance(v, total_sessions)
        )
    else:
        merged_df['% Asistencia'] = pd.NA
    
    # Reorder columns: pseudonym and role first
    leading_cols = ['Nombre_pseudonimo', 'Es_profesor']
    other_cols = [c for c in merged_df.columns if c not in leading_cols]
    merged_df = merged_df[leading_cols + other_cols]
    
    # Drop sensitive columns
    sensitive_cols = [
        'Nombre completo del usuario',
        'Usuario afectado',
        'Dirección IP',
        'Origen'
    ]
    merged_df = merged_df.drop(
        columns=[c for c in sensitive_cols if c in merged_df.columns],
        errors='ignore'
    )
    
    # Summary statistics
    n_students = sum(v.startswith('STU_') for v in pseudonym_mapping.values())
    n_teachers = sum(v.startswith('TEACHER_') for v in pseudonym_mapping.values())
    
    print(f"Final merged dataset: {len(merged_df)} rows")
    print(f"Students pseudonymized: {n_students}")
    print(f"Teachers identified: {n_teachers}")
    
    # Save output if path provided
    if output_file:
        merged_df.to_csv(output_file, index=False)
        print(f"Saved to: {output_file}")
    
    return merged_df, pseudonym_mapping

## 4. Batch Processing Configuration

This section configures the batch processing of multiple academic years. The configuration includes:

- **Years**: List of academic year suffixes to process
- **Per-year parameters**: Cutoff dates and session counts
- **File templates**: Naming patterns for input and output files
- **Teacher roster**: Names of instructors (for labeling, not pseudonymization)
- **Study salt**: Secret value for cryptographic hashing (kept confidential)

### Important Privacy Note

The `STUDY_SALT` variable should be replaced with a strong, unique secret value for your study. This salt:
- Must be kept confidential
- Should never be logged, printed, or saved to output files
- Enables consistent pseudonyms across processing runs
- Prevents dictionary-based re-identification attacks

In [6]:
# ==============================================================================
# BATCH CONFIGURATION
# ==============================================================================

# Academic years to process (suffix used in file names)
YEARS = ['22', '23', '24', '25']

# Per-year configuration: cutoff date and total session count
YEAR_CONFIG = {
    '22': {'limit': '22/05/2022', 'sessions': 32},
    '23': {'limit': '15/05/2023', 'sessions': 30},
    '24': {'limit': '23/05/2024', 'sessions': 37},
    '25': {'limit': '20/05/2025', 'sessions': 30}
}

# File naming templates (adjust paths as needed for your directory structure)
MOODLE_TEMPLATE = 'moodle_{year}.csv'
EVAL_TEMPLATE = 'evaluacion_{year}.csv'
OUTPUT_TEMPLATE = 'merged_data_20{year}.csv'

# Teacher roster for identification (case and accent insensitive matching)
TEACHERS = [
    'Alicia García Holgado',
    'Andrea Vázquez Ingelmo',
    'Francisco José García Peñalvo'
]


STUDY_SALT = ''

## 5. Batch Execution

The following cell executes the merge function for each academic year. For each year it:

1. Verifies that input files exist
2. Calls the merge function with year-specific parameters
3. Adds a `Year` column for provenance tracking
4. Stores results in memory for aggregation
5. Writes anonymized per-year CSV files

If a year's files are missing or processing fails, the error is logged and execution continues with the next year.

In [7]:
# Storage for results across all years
all_merged_data = {}
all_name_mappings = {}

for year_suffix in YEARS:
    print(f"\n{'='*60}")
    print(f"Processing academic year 20{year_suffix}")
    print('='*60)
    
    # Construct file paths
    moodle_file = Path(MOODLE_TEMPLATE.format(year=year_suffix))
    evaluation_file = Path(EVAL_TEMPLATE.format(year=year_suffix))
    output_file = OUTPUT_TEMPLATE.format(year=year_suffix)
    
    # Validate input files exist
    if not moodle_file.exists():
        print(f"[SKIP] Missing Moodle file: {moodle_file}")
        continue
    if not evaluation_file.exists():
        print(f"[SKIP] Missing Evaluation file: {evaluation_file}")
        continue
    
    try:
        # Execute the merge pipeline
        merged_df, name_mapping = merge_moodle_evaluation_data(
            moodle_file=str(moodle_file),
            evaluation_file=str(evaluation_file),
            total_sessions=YEAR_CONFIG[year_suffix]['sessions'],
            limit_date=YEAR_CONFIG[year_suffix]['limit'],
            teachers=TEACHERS,
            output_file=str(output_file),
            manual_matches=None,  # Add manual overrides here if needed
            fuzzy_threshold=0.60,
            suggestion_threshold=0.50,
            salt=STUDY_SALT
        )
        
        # Add provenance column for multi-year analyses
        merged_df['Year'] = f"20{year_suffix}"
        
        # Store results
        all_merged_data[f"20{year_suffix}"] = merged_df
        all_name_mappings[f"20{year_suffix}"] = name_mapping
        
        print(f"\n[SUCCESS] Year 20{year_suffix}: {len(merged_df)} rows -> {output_file}")
        
    except Exception as e:
        print(f"\n[ERROR] Year 20{year_suffix} failed: {e}")
        continue


Processing academic year 2022
Loading datasets...
Filtering Moodle actions after 22/05/2022 (inclusive end).
Moodle rows kept: 44535 / 54771
Unique names in Moodle: 75
Unique names in Evaluation: 72
Exact matches found: 69
Finding fuzzy matches...
Fuzzy match: 'ALIAS_1' -> 'ALIAS_2' (score: 0.935)
Fuzzy match: 'ALIAS_3' -> 'ALIAS_4' (score: 0.957)
Total matches found after fuzzy: 71

Unmatched in Moodle logs (4): ['ALIAS_5', 'ALIAS_6', 'ALIAS_7', 'ALIAS_8']

Unmatched in Moodle logs (1): ['ALIAS_9']

Suggested matches (>=50% similarity):

Filtering unmatched non-teacher records...
Original Moodle records: 44535
Records removed per user (masked): {'ALIAS_10': 64, 'ALIAS_11': 505, 'ALIAS_8': 8}
Filtered Moodle records: 43958

Merging datasets...
Pseudonymizing names...
Final merged dataset: 43958 rows
Students pseudonymized: 71
Teachers identified: 3
Saved to: merged_data_2022.csv

[SUCCESS] Year 2022: 43958 rows -> merged_data_2022.csv

Processing academic year 2023
Loading datasets...

## 6. Combine All Years

This cell concatenates all per-year datasets into a single combined file for longitudinal analyses. The `Year` column enables filtering by academic year as needed.

In [8]:
if all_merged_data:
    combined_df = pd.concat(all_merged_data.values(), ignore_index=True, sort=False)
    combined_path = 'merged_data_all_years.csv'
    combined_df.to_csv(combined_path, index=False)
    
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Saved combined CSV to: {combined_path}")
else:
    combined_df = pd.DataFrame()
    print("No yearly datasets were produced; combined dataset is empty.")

Combined dataset shape: (214730, 17)
Saved combined CSV to: merged_data_all_years.csv


## 7. Processing Summary

The following table summarizes the pseudonymization results for each academic year, showing the number of students and teachers processed.

In [9]:
summary_data = []
for year, mapping in all_name_mappings.items():
    n_total = len(mapping)
    n_teachers = sum(1 for v in mapping.values() if str(v).startswith('TEACHER_'))
    n_students = n_total - n_teachers
    summary_data.append((year, n_students, n_teachers, n_total))

summary_df = pd.DataFrame(
    summary_data,
    columns=['Year', 'Students (pseudonymized)', 'Teachers (labeled)', 'Total mapped']
)

display(summary_df)

,Year,Students (pseudonymized),Teachers (labeled),Total mapped
0,2022,71,3,74
1,2023,71,2,73
2,2024,63,2,65
3,2025,79,2,81


## 8. Reproducibility Information

The following cell prints environment information for reproducibility documentation. This can be included in the methods section of publications to ensure the analysis is auditable.

In [10]:
print(f"Processing timestamp: {datetime.now().isoformat(timespec='seconds')}")
print(f"pandas version: {pd.__version__}")
print(f"Python hashlib: SHA-256 (salted)")

Processing timestamp: 2025-12-11T13:15:31
pandas version: 2.3.3
Python hashlib: SHA-256 (salted)


## Appendix: Column Descriptions

### Output Dataset Columns

| Column | Type | Description |
|--------|------|-------------|
| `Nombre_pseudonimo` | str | Anonymized identifier (STU_XXXXXXXX or TEACHER_N) |
| `Es_profesor` | bool | True if the record belongs to an instructor |
| `Hora` | str | Timestamp of the Moodle activity |
| `Contexto del evento` | str | Moodle context (course, module) |
| `Componente` | str | Moodle component generating the event |
| `Nombre evento` | str | Type of activity performed |
| `Descripción` | str | Detailed event description |
| `Hito 1-3` | float | Milestone grades (if available) |
| `Trabajo` | float | Project/assignment grade |
| `Coevaluación 1-3` | float | Peer evaluation scores |
| `% Asistencia` | float | Attendance percentage [0-100] |
| `Year` | str | Academic year (e.g., "2022") |